# Meraki Python SDK Demo: Uplink Preference Backup

*This notebook demonstrates using the Meraki Python SDK to back up Internet (WAN) and VPN traffic uplink preferences, as well as custom performance classes, to a single Excel file. If you have hundreds of WAN/VPN uplink preferences, they can be a challenge to manipulate. This demo seeks to prove how using the Meraki API and Python SDK can substantially streamline such complex deployments.*

If an admin has any Internet traffic or VPN traffic uplink preferences, or custom performance classes, this tool will download them to an Excel file. This is a more advanced demo, intended for intermediate to advanced Python programmers, but has been documented thoroughly with the intention that even a determined Python beginner can understand the concepts involved.

---

>NB: Throughout this notebook, we will print values for demonstration purposes. In a production Python script, the coder would likely remove these print statements to clean up the console output.

In this first cell, we import the required `meraki` and `os` modules, and open the Dashboard API connection using the SDK. We also import `openpyxl` for working with Excel files, and `netaddr` for working with IP addresses.

In [ ]:
# Install the relevant modules. If you are using a local editor (e.g. VS Code, rather than Colab) you can run these commands, without the preceding %, via a terminal. NB: Run `pip install meraki==` to find the latest version of the Meraki SDK. Uncomment these lines to run them in Google Colab.
#%pip install meraki
#%pip install openpyxl

# If you are using Google Colab, please ensure you have set up your environment variables as linked above, then delete the two lines of ''' to activate the following code:
'''
%pip install colab-env -qU
import colab_env
'''

# The Meraki SDK
import meraki
# The built-in OS module, to read environment variables
import os
# We're also going to import Python's built-in JSON module, but only to make the console output pretty. In production, you wouldn't need any of the printing calls at all, nor this import!
import json

# The openpyxl module, to manipulate Excel files
import openpyxl
# The datetime module, to generate timestamps
import datetime

# Treat your API key like a password. Store it in your environment variables as 'MERAKI_DASHBOARD_API_KEY' and let the SDK call it for you.
# Or, call it manually after importing Python's os module:
# API_KEY = os.getenv('MERAKI_DASHBOARD_API_KEY')

# Initialize the Dashboard connection.
dashboard = meraki.DashboardAPI(suppress_logging=True)

# We'll also create a few reusable strings for later interactivity.
string_constants = dict()
string_constants['CONFIRM'] = 'OK, are you sure you want to do this? This script does not have an "undo" feature.'
string_constants['CANCEL'] = 'OK. Operation canceled.'
string_constants['WORKING'] = 'Working...'
string_constants['COMPLETE'] = 'Operation complete.'
string_constants['NETWORK_SELECTED'] = 'Network selected.'
string_constants['NO_VALID_OPTIONS'] = 'There are no valid options. Please try again with an API key that has access to the appropriate resources.'

# Some of the parameters we'll work with are optional. This string defines what value will be put into a cell corresponding with a parameter that is not set on that rule.
string_constants['NOT_APPLICABLE'] = 'N/A'

# This script is interactive; user choices and data will be stored here.
user_choices = dict()
user_data = dict()

A basic pretty print formatter, `printj()`. It will make reading JSON on the console easier, but won't be necessary in production scripts.

In [ ]:
def printj(ugly_json_object):
    
     # The json.dumps() method converts a JSON object into human-friendly formatted text
    pretty_json_string = json.dumps(ugly_json_object, indent = 2, sort_keys = False)
    
    return print(pretty_json_string)

## Introducing a Python class

To streamline user interaction in a re-usable way, we'll create a class called UserChoice. Think of classes like a superset of functions, where you can store related functions and variables. Later, we'll create an instance of this class to prompt the user for input, and validate that input. 

It may look complex, but it will streamline our code later, and is a great example of code-reuse in Python. For more information on classes, [click here](https://docs.python.org/3/tutorial/classes.html). Alternatively, you could reduce the script's interactivity and have it read required settings from a settings file, as in [this example](https://github.com/meraki/dashboard-api-python/tree/master/examples/merakiApplianceVlanToL3SwitchInterfaceMigrator).

In [ ]:
class UserChoice:

    'A re-usable CLI option prompt.'

    def __init__(self, options_list=[], subject_of_choice='available options', single_option_noun='option', id_parameter='id', name_parameter='name', action_verb='choose', no_valid_options_message=string_constants['NO_VALID_OPTIONS']):

        self.options_list = options_list # options_list is a list of dictionaries containing attributes id_parameter and name_parameter  
        self.subject_of_choice = subject_of_choice # subject_of_choice is a string that names the subject of the user's choice. It is typically a plural noun.
        self.single_option_noun = single_option_noun # single_option_noun is a string that is a singular noun corresponding to the subject_of_choice
        self.id_parameter = id_parameter # id_parameter is a string that represents the name of the sub-parameter that serves as the ID value for the option in options_list. It should be a unique value for usability.
        self.name_parameter = name_parameter # name_paraemter is a string that represents the name of the sub-parameter that serves as the name value for the option in options_list. It does not need to be unique.
        self.action_verb = action_verb # action_verb is a string that represents the verb of the user's action. For example, to "choose"
        self.no_valid_options_message = no_valid_options_message # no_valid_options_message is a string that represents an error message if options_list is empty
              
        # Confirm there are options in the list
        if len(self.options_list):
            print(f'We found {len(self.options_list)} {self.subject_of_choice}:')

            # Label each option and show the user their choices.
            option_index = 0

            for option in self.options_list:
                print(f"{option_index}. {option[self.id_parameter]} with name {option[self.name_parameter]}")
                option_index+=1

            print(f'Which {self.single_option_noun} would you like to {self.action_verb}?')
            self.active_option = int(input(f'Choose 0-{option_index-1}:'))
            
            # Ask until the user provides valid input.
            while self.active_option not in list(range(option_index)):
                print(f'{self.active_option} is not a valid choice. Which {self.single_option_noun} would you like to {self.action_verb}?')
                self.active_option = int(input(f'Choose 0-{option_index-1}:'))

            print(f'Your {self.single_option_noun} is {self.options_list[self.active_option][self.name_parameter]}.')

            # Assign the class id and name vars to the chosen item's
            self.id = self.options_list[self.active_option][self.id_parameter]
            self.name = self.options_list[self.active_option][self.name_parameter]

## Pulling organization and network IDs

Most API calls require passing values for the organization ID and/or the network ID. Remember that `UserChoice` class we created earlier? We'll call that and supply parameters defining what the user can choose. Notice how, having defined the class earlier, we can re-use it with only a single declaration.

In [ ]:
# getOrganizations will return all orgs to which the supplied API key has access
user_choices['all_organizations'] = dashboard.organizations.getOrganizations()

# Prompt the user to pick an organization.
user_choices['organization'] = UserChoice(
    options_list=user_choices['all_organizations'], 
    subject_of_choice='organizations', 
    single_option_noun='organization'
    )

## Identify networks with MX appliances, and prompt the user to choose one

We want to:
> Download any existing uplink selection preferences and custom performance classes. 

> Optionally, upload a replacement list of preferences.

We can only run this on networks that have appliance devices, so we will find networks where `productTypes` contains `appliance`. Then we'll ask the user to pick one, then pull the uplink selection rules from it.

Then let's ask the user which network they'd like to use.



In [ ]:
# Make a list of all networks in the org
user_choices['all_networks'] = dashboard.organizations.getOrganizationNetworks(organizationId=user_choices['organization'].id)

# Find the networks with appliances
user_choices['networks_with_appliances'] = [network for network in user_choices['all_networks'] if 'appliance' in network['productTypes']]

# If any are found, let the user choose a network. Otherwise, let the user know that none were found. The logic for this class is defined in a cell above.
user_choices['network'] = UserChoice(
    options_list = user_choices['networks_with_appliances'], 
    subject_of_choice = 'networks with appliances', 
    single_option_noun = 'network'
    )

## Pulling uplink preferences for the network

Let's pull the uplink preferences via the API using the SDK's `getNetworkApplianceTrafficShapingUplinkSelection` and `getNetworkApplianceTrafficShapingCustomPerformanceClasses` methods. The associated endpoints will return all of the relevant information for the chosen network. We'll create an Excel file of it later.

Let's define this operation as a function so we can re-use it later.

In [ ]:
def getNetworkClassesAndPrefs(*, networkId):
    # Get the uplink preferences (WAN and VPN)
    uplink_prefs = dashboard.appliance.getNetworkApplianceTrafficShapingUplinkSelection(networkId=networkId)
    
    # Get the custom performance classes
    custom_performance_classes = dashboard.appliance.getNetworkApplianceTrafficShapingCustomPerformanceClasses(networkId=networkId)
    
    # Create a dict network_classes_and_prefs with both that we can return and call later
    network_classes_and_prefs = {
        'uplinkPrefs': uplink_prefs, 
        'customPerformanceClasses': custom_performance_classes
        }
    return(network_classes_and_prefs)

user_data['current_classes_and_prefs'] = getNetworkClassesAndPrefs(networkId = user_choices['network'].id)

The above returns all uplink preferences (e.g. Internet traffic and VPN traffic). We can review the output of each individually if we'd like to see the raw data:

In [ ]:
# We can review the wanTrafficUplinkPreferences in plain text.
printj(user_data['current_classes_and_prefs']['uplinkPrefs']['wanTrafficUplinkPreferences'])

In [ ]:
# We can also review the vpnTrafficUplinkPreferences in plain text.
printj(user_data['current_classes_and_prefs']['uplinkPrefs']['vpnTrafficUplinkPreferences'])

In [ ]:
# We can also review the customPerformanceClasses in plain text.
printj(user_data['current_classes_and_prefs']['customPerformanceClasses'])

## Let's make a backup

Before we start modifying any of these rules or classes, it'd be good to make a backup, don't you think? We'll use `openpyxl` to create a new Excel workbook with two worksheets: one for Internet traffic preferences and the other for VPN traffic preferences. If you don't care to use Excel, you could instead create a CSV or potentially any other type of file you like, using an appropriate Python module. Consider a self-paced excercise where you save these values into some other file format.

We'll start with the creation of the workbook. We'll make this a function for easy calling later.

In [ ]:
def create_workbook():

    # Create a workbook with the appropriate column headers and/or worksheets
    # First we'll create the workbook, then we'll design the worksheets. Just like in Excel, by default, the workbook has one worksheet, titled 'Sheet'.
    new_workbook = openpyxl.Workbook()

    # We'll specify that the active worksheet is our wan_prefs_worksheet
    wan_prefs_worksheet = new_workbook.active

    # Let's rename the worksheet from 'Sheet' to something more descriptive
    wan_prefs_worksheet.title = 'wanUplinkPreferences'

    # Name the columns for the wan_prefs_worksheet. Think of this like a single-line CSV:
    wan_title_row_headers = (
        'Protocol',
        'Source',
        'Src port',
        'Destination',
        'Dst port',
        'Preferred uplink'
    )

    # Add that title row to the worksheet
    wan_prefs_worksheet.append(wan_title_row_headers)

    # Let's make the title row bold for easier reading
    for row in wan_prefs_worksheet.iter_rows():
        for cell in row:
            cell.font = openpyxl.styles.Font(bold=True)

    # Now let's do the same for the VPN uplink preferences, and custom performance classes.
    # First, create a separate worksheet for the VPN uplink preferences
    vpn_prefs_worksheet = new_workbook.create_sheet(title='vpnUplinkPreferences')
    custom_performance_classes_worksheet = new_workbook.create_sheet(title='customPerformanceClasses')

    # Name the columns for the vpn_prefs_worksheet and custom_performance_classes_worksheet.
    vpn_title_row_headers = (
        'Type',
        'Protocol or App ID',
        'Source or App Name',
        'Src port',
        'Destination',
        'Dst port',
        'Preferred uplink',
        'Failover criterion',
        'Performance class type',
        'Performance class name',
        'Performance class ID'
    )

    classes_title_row_headers = (
        'ID',
        'Name',
        'Max Latency',
        'Max Jitter',
        'Max Loss Percentage'
    )

    # Add the title rows to the appropriate worksheets
    vpn_prefs_worksheet.append(vpn_title_row_headers)
    custom_performance_classes_worksheet.append(classes_title_row_headers)

    # Let's make those title rows bold, too
    for row in vpn_prefs_worksheet.iter_rows():
        for cell in row:
            cell.font = openpyxl.styles.Font(bold=True)
    
    for row in custom_performance_classes_worksheet.iter_rows():
        for cell in row:
            cell.font = openpyxl.styles.Font(bold=True)

    print(f'Created formatted workbook.')
    
    return(new_workbook)

### Function to add custom performance classes to a workbook

VPN uplink prefs might reference performance classes. Accordingly, we need to back those up as well.

In [ ]:
def add_custom_performance_classes_to_workbook(workbook):

    # We'll specify that the active worksheet is our custom_performance_classes worksheet
    custom_performance_classes_worksheet = workbook['customPerformanceClasses']

    # We'll also count the number of classes to help the user know that it's working.
    performance_class_count = 0

    # Let's add all the vpnTrafficUplinkPreferences to the VPN worksheet
    for performance_class in user_data['current_classes_and_prefs']['customPerformanceClasses']:

        performance_class_id = performance_class['customPerformanceClassId']
        performance_class_name = performance_class['name']
        performance_class_max_latency = performance_class['maxLatency']
        performance_class_max_jitter = performance_class['maxJitter']
        performance_class_max_loss_percentage = performance_class['maxLossPercentage']

        # We assemble the parameters into a tuple that will represent a row.
        performance_class_row = (
            performance_class_id,
            performance_class_name,
            performance_class_max_latency,
            performance_class_max_jitter,
            performance_class_max_loss_percentage
            )

        # We then add that row to the worksheet.
        custom_performance_classes_worksheet.append(performance_class_row)

        # increase the rule_count
        performance_class_count += 1

    
    print(f'Added {performance_class_count} performance classes to customPerformanceClasses.')
    
    return(workbook)

### Function to add WAN preferences to a workbook

Transposing the settings into a two-dimensional table doesn't require anything too fancy. We simply iterate through the rules in the `uplinkPrefs['wanTrafficUplinkPreferences']` list, and pull out the relevant information for each key-value pair. We'll also make this a function so we can call it later.

In [ ]:
def add_wan_prefs_to_workbook(workbook):

    # We'll specify that the active worksheet is our wan_prefs_worksheet
    wan_prefs_worksheet = workbook['wanUplinkPreferences']

    # We'll also count the number of rules to help the user know that it's working.
    rule_count = 0

    # Let's add all the wanTrafficUplinkPreferences
    for rule in user_data['current_classes_and_prefs']['uplinkPrefs']['wanTrafficUplinkPreferences']:
        
        rule_preferred_uplink = rule['preferredUplink']   
        rule_protocol = rule['trafficFilters'][0]['value']['protocol']
        rule_source = rule['trafficFilters'][0]['value']['source']['cidr']

        # An 'any' value in the protocol removes the need for either source or destination port numbers to be defined, so the API doesn't specify port numbers in the output if 'any' is the protocol. However, we don't want to leave those cells blank, so we will fill them in accordingly.
        
        if rule_protocol == 'any':
            rule_src_port = 'any'
            rule_dst_port = 'any'
        else:
            rule_src_port = rule['trafficFilters'][0]['value']['source']['port']
            rule_dst_port = rule['trafficFilters'][0]['value']['destination']['port']

        rule_destination = rule['trafficFilters'][0]['value']['destination']['cidr']


        # We assemble the parameters into a tuple that will represent a row.
        rule_row = (
            rule_protocol, 
            rule_source, 
            rule_src_port, 
            rule_destination, 
            rule_dst_port, 
            rule_preferred_uplink
            )

        # We then add that row to the worksheet.
        wan_prefs_worksheet.append(rule_row)
        
        # increase the rule_count
        rule_count += 1

    print(f'Added {rule_count} rules to wanUplinkPreferences.')
    
    return(workbook)

### Function to add VPN preferences to a workbook

We'll do almost the exact same for the VPN traffic uplink preferences, but since there are more parameters available here, we employ a bit more `if`/`else` logic to handle the varied key structures.

In [ ]:
def add_vpn_prefs_to_workbook(workbook):

    # We'll specify that the active worksheet is our wan_prefs_worksheet
    vpn_prefs_worksheet = workbook['vpnUplinkPreferences']

    # We'll also count the number of rules to help the user know that it's working.
    rule_count = 0

    # Let's add all the vpnTrafficUplinkPreferences to the VPN worksheet
    for rule in user_data['current_classes_and_prefs']['uplinkPrefs']['vpnTrafficUplinkPreferences']:

        rule_preferred_uplink = rule['preferredUplink']
        rule_type = rule['trafficFilters'][0]['type']

        # Application rules have different parameters. This checks, and assigns values accordingly.
        if 'application' in rule_type:
            rule_protocol = rule['trafficFilters'][0]['value']['id']
            rule_source = rule['trafficFilters'][0]['value']['name']
            rule_src_port = string_constants['NOT_APPLICABLE']
            rule_destination = string_constants['NOT_APPLICABLE']
            rule_dst_port = string_constants['NOT_APPLICABLE']
        else: 
            rule_protocol = rule['trafficFilters'][0]['value']['protocol']
            rule_source = rule['trafficFilters'][0]['value']['source']['cidr']
            rule_destination = rule['trafficFilters'][0]['value']['destination']['cidr']

            # An 'any' or 'icmp' value in the protocol removes the need for either source or destination port numbers to be defined, so the API doesn't specify port numbers in the output if 'any' or 'icmp' is the protocol. However, we don't want to leave those cells blank, so we will fill them in accordingly.
            
            if rule_protocol == 'any':
                rule_src_port = 'any'
                rule_dst_port = 'any'
            elif rule_protocol == 'icmp':
                rule_src_port = string_constants['NOT_APPLICABLE']
                rule_dst_port = string_constants['NOT_APPLICABLE']
            else:
                rule_src_port = rule['trafficFilters'][0]['value']['source']['port']
                rule_dst_port = rule['trafficFilters'][0]['value']['destination']['port']
        
        # A failover criterion and performance class are optional parameters, so we cannot assume they are there. We'll first check to see if they exist before pulling their values.
        # Check if the rule has failOverCriterion set
        if rule.get('failOverCriterion', False):
            rule_failover_criterion = rule['failOverCriterion']
        else: # No failOverCriterion set
            rule_failover_criterion = string_constants['NOT_APPLICABLE']

        # Check if the rule has performanceClass set
        if rule.get('performanceClass', False):
            rule_performance_class_type = rule['performanceClass']['type']

            # If the performance class is set, and is 'builtin', then we use 'builtinPerformanceClassName'. If it's 'custom', then we use 'customPerformanceClassId' to identify it.
            if rule_performance_class_type == 'builtin':
                rule_performance_class_id = string_constants['NOT_APPLICABLE']
                rule_performance_class_name = rule['performanceClass']['builtinPerformanceClassName']
            else:
                rule_performance_class_id = rule['performanceClass']['customPerformanceClassId']
                
                # search user_data['current_classes_and_prefs']['customPerformanceClasses'] for the class that has that ID, then pull the corresponding name
                for performance_class in user_data['current_classes_and_prefs']['customPerformanceClasses']: 
                    if rule_performance_class_id == performance_class['customPerformanceClassId']:
                        rule_performance_class_name = performance_class['name']
                
                
        # Else, there's no performanceClass set, so we'll set these values accordingly.
        else:
            rule_performance_class_type = string_constants['NOT_APPLICABLE']
            rule_performance_class_name = string_constants['NOT_APPLICABLE']
            rule_performance_class_id = string_constants['NOT_APPLICABLE']

        # We assemble the parameters into a tuple that will represent a row.
        rule_row = (
            rule_type,
            rule_protocol, 
            rule_source, 
            rule_src_port, 
            rule_destination, 
            rule_dst_port, 
            rule_preferred_uplink,
            rule_failover_criterion,
            rule_performance_class_type,
            rule_performance_class_name,
            rule_performance_class_id
            )

        # We then add that row to the worksheet.
        vpn_prefs_worksheet.append(rule_row)

        # increase the rule_count
        rule_count += 1

    
    print(f'Added {rule_count} rules to wanUplinkPreferences.')
    
    return(workbook)

### Function to save a workbook

This function takes a single workbook as an argument, and saves it.

In [ ]:
# Function that saves a workbook
def save_workbook(workbook):

    # Finally, we save the worksheet.
    # Let's give it a name with a date/time stamp
    downloaded_rules_workbook_filename = f'downloaded_rules_workbook_{datetime.datetime.now()}.xlsx'.replace(':','')

    workbook.save(downloaded_rules_workbook_filename)

    print(f'Saved {downloaded_rules_workbook_filename}.')

### Build and save the workbook

Now that we've defined the functions that will build the workbook object in Python and save it to a file, we need to run them.

In [ ]:
uplink_prefs_workbook = create_workbook()
add_custom_performance_classes_to_workbook(uplink_prefs_workbook)
add_wan_prefs_to_workbook(uplink_prefs_workbook)
add_vpn_prefs_to_workbook(uplink_prefs_workbook)
save_workbook(uplink_prefs_workbook)

# Final thoughts

Hopefully this was a useful deep dive into Python programming and interacting with the Meraki SDK and Excel workbooks. We tackled a problem that is tough to solve in the Dashboard GUI and showed how it can be done very quickly via API and the Python SDK.

But what if we want to RESTORE that backup? Well, that's the next challenge! [Return to the notebooks folder](https://github.com/meraki/dashboard-api-python/tree/master/notebooks) for an example notebook that can restore such a backup to the Dashboard.

Here we used Excel workbooks, but you can imagine that there are all types of data structures that might be used instead of Excel workbooks, e.g. CSVs, plain text, YAML, XML, LibreOffice files or others, and with the right code you can use those instead of Excel.

## Further learning

[Meraki Interactive API Docs](https://developer.cisco.com/meraki/api-v1/#!overview): The official (and interactive!) Meraki API and SDK documentation repository on DevNet.